In [1]:
import SimpleITK as sitk
import numpy as np
import csv
from glob import glob
import pandas as pd

# path constants
LUNA_DATA_PATH = '../../../../data/luna16/'
LUNA_SUBSET_PATH = LUNA_DATA_PATH + 'subset0/'

file_list = glob(LUNA_SUBSET_PATH + "*.mhd") # get all the mhd image files

# Helper function to get rows in data frame associated with each file
def get_filename(case):
    global file_list
    for f in file_list: # for every file in the list if the seriesuid is in the file name, return the file 
        if case in f:
            return(f)

# The locations of the nodes
df_node = pd.read_csv(LUNA_DATA_PATH + "annotations.csv")
df_node["file"] = df_node["seriesuid"].apply(get_filename) # for every rowsave file name to the 'file' column of the row
df_node = df_node.dropna() # if the seriesuid is not found in this subset, drop all the rows that have na as values for 'file' column


mini_df = df_node[df_node["file"]==file_list[2]]
print(mini_df)
print('\n')
print(mini_df["diameter_mm"].values)
print('\n')
print(np.argsort(mini_df["diameter_mm"].values))
print('\n')
print(np.argsort(mini_df["diameter_mm"].values)[-1])

                                            seriesuid     coordX     coordY  \
25  1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524...  46.188539  48.402806   
26  1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524...  36.392044  76.771663   

        coordZ  diameter_mm                                               file  
25 -108.578632    13.596471  ../../../../data/luna16/subset0\1.3.6.1.4.1.14...  
26 -123.321911     4.343200  ../../../../data/luna16/subset0\1.3.6.1.4.1.14...  


[ 13.59647134   4.34319974]


[1 0]


0


In [2]:
img_file = file_list[2]
itk_img = sitk.ReadImage(img_file) 
print(itk_img)

Image (00000214B1B0DA50)
  RTTI typeinfo:   class itk::Image<short,3>
  Reference Count: 1
  Modified Time: 1060
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 1035
  UpdateMTime: 1059
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 161]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 161]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 161]
  Spacing: [0.548828, 0.548828, 1.25]
  Origin: [-187.7, -108.3, -194]
  Direction: 
1 0 0
0 1 0
0 0 1

  IndexToPointMatrix: 
0.548828 0 0
0 0.548828 0
0 0 1.25

  PointToIndexMatrix: 
1.82206 0 0
0 1.82206 0
0 0 0.8

  Inverse Direction: 
1 0 0
0 1 0
0 0 1

  PixelContainer: 
    ImportImageContainer (00000214B1C224D0)
      RTTI typeinfo:   class itk::ImportImageContainer<unsigned __int

In [3]:
img_array = sitk.GetArrayFromImage(itk_img)
print(img_array)

[[[-3024 -3024 -3024 ..., -3024 -3024 -3024]
  [-3024 -3024 -3024 ..., -3024 -3024 -3024]
  [-3024 -3024 -3024 ..., -3024 -3024 -3024]
  ..., 
  [-3024 -3024 -3024 ..., -3024 -3024 -3024]
  [-3024 -3024 -3024 ..., -3024 -3024 -3024]
  [-3024 -3024 -3024 ..., -3024 -3024 -3024]]

 [[-3024 -3024 -3024 ..., -3024 -3024 -3024]
  [-3024 -3024 -3024 ..., -3024 -3024 -3024]
  [-3024 -3024 -3024 ..., -3024 -3024 -3024]
  ..., 
  [-3024 -3024 -3024 ..., -3024 -3024 -3024]
  [-3024 -3024 -3024 ..., -3024 -3024 -3024]
  [-3024 -3024 -3024 ..., -3024 -3024 -3024]]

 [[-3024 -3024 -3024 ..., -3024 -3024 -3024]
  [-3024 -3024 -3024 ..., -3024 -3024 -3024]
  [-3024 -3024 -3024 ..., -3024 -3024 -3024]
  ..., 
  [-3024 -3024 -3024 ..., -3024 -3024 -3024]
  [-3024 -3024 -3024 ..., -3024 -3024 -3024]
  [-3024 -3024 -3024 ..., -3024 -3024 -3024]]

 ..., 
 [[-3024 -3024 -3024 ..., -3024 -3024 -3024]
  [-3024 -3024 -3024 ..., -3024 -3024 -3024]
  [-3024 -3024 -3024 ..., -3024 -3024 -3024]
  ..., 
  [-3024 -

In [9]:
biggest_node = np.argsort(mini_df["diameter_mm"].values)[-1]
node_x = mini_df["coordX"].values[biggest_node]
node_y = mini_df["coordY"].values[biggest_node]
node_z = mini_df["coordZ"].values[biggest_node]
diam = mini_df["diameter_mm"].values[biggest_node]
print(biggest_node)
print(node_x)
print(node_y)
print(node_z)
print(diam)
print('\n')

center = np.array([node_x,node_y,node_z])   # nodule center
origin = np.array(itk_img.GetOrigin())      # x,y,z  Origin in world coordinates (mm)
spacing = np.array(itk_img.GetSpacing())    # spacing of voxels in world coor. (mm)
v_center =np.rint((center-origin)/spacing) 

print(itk_img.GetOrigin())
print(itk_img.GetSpacing())

0
46.18853869
48.40280596
-108.5786324
13.59647134


(-187.699997, -108.300003, -194.0)
(0.5488280057907104, 0.5488280057907104, 1.25)
